# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#store file path
weather_file = "../WeatherPy/output.csv"
#read file
weather_df = pd.read_csv(weather_file, encoding="ISO-8859-1")
#show the header
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,show low,34.2542,-110.0298,41.00,24,1,9.22,US,1618645100
1,upernavik,72.7868,-56.1549,-10.37,93,79,6.60,GL,1618645101
2,barrow,71.2906,-156.7887,23.00,80,90,12.66,US,1618645101
3,lebu,-37.6167,-73.6500,52.14,82,24,4.63,CL,1618645102
4,kaitong,44.8135,123.1500,53.29,34,74,12.35,CN,1618645102


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#use lat and lng and humidity
locations = weather_df[["Lat", "Lng"]]

humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#Narrow down the cities to cities that fill ideal conditions
ideal_weather = weather_df[(70 < weather_df['Max Temp'])
                                           &(weather_df['Max Temp'] <80) 
                                           & (weather_df['Wind Speed'] < 10) 
                                           & (weather_df['Cloudiness'] == 0)]
ideal_weather


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
129,beroroha,-21.6667,45.1667,78.17,36,0,7.43,MG,1618645152
145,conceicao do araguaia,-8.2578,-49.2647,75.20,100,0,1.14,BR,1618645158
218,itanhaem,-24.1831,-46.7889,73.00,72,0,5.75,BR,1618645191
234,safut,32.0336,35.8332,75.20,23,0,2.30,JO,1618645197
274,plettenberg bay,-34.0527,23.3716,73.74,26,0,1.54,ZA,1618645213
514,hasanabad,36.5465,61.0543,77.00,38,0,9.22,IR,1618645301


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
ideal_weather

ideal_weather["Hotel Name"] = ""

<ipython-input-9-3ac8aae0bc56>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather["Hotel Name"] = ""


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
